#Team 5 - Learning Models by Exploiting Structure and Fundamental Knowledge

Stuff to add

*   Background on how to take a gradient
*   tell a story to justify why we are implementing these loss functions
*   ground each function in what part of the state representation we want to learn



In [ ]:
from checker import check_solution
import solutions
import numpy as np

# Problem 1 - Loss Functions

In lecture, we stepped through multiple components of a loss function that is used to learn physics-based state representations. However, we do not use the loss function directly during learning, but rather the gradient of the loss function to perform gradient descent. In this problem, you will implement the gradient of each component of the loss function.

The incoming data will be formatted as an array of Data_Frame objects which are defined in utils_pset. At a high level, these objects have:



*   `action`: an integer indicating the action taken at the given timestep (up, down, left right)
*   `reward`: the reward received at the given timestep for moving away from a position.
*   `image`: a flattened array of the pixel values of the image taken of the agent (the robot on the grid)

Using the incoming data as well as the matrix used to transform image vectors to coordinates, you will step through and implement the loss function defined in lecture that biases state representations toward following Newton's laws of physics. Each section will step you through implementing a different component of the loss function: temporal cohesion, proportionality, causality, repeatability. These are the physical prior of the environment for this representation.

The mapping matrix is the matrix we are learning to transform high dimensional image vectors to low dimensional state representations. Wherever we use the variable $s_t$ to represent a state, this can be thought of as $s_t = m v_t$, where $m$ is the mapping matrix that we are learning, and $v_t$ is the image vector taken of the world at time $t$.




## Worked Example

First, we want our agent to learn state representations that follow the rule that F=ma. This means we need to implement the Proportionality Prior Gradient.

The proportionality prior loss is defined as $E[(||\triangle s_{t_2}|| - ||\triangle s_{t_1}||)^2 | a_{t_1} = a_{t_2}]$, for two time steps $t_1$ and $t_2$. 

We have implemented this for you to give you an example of how to implementing the remaining loss function components. We have broken the gradient into two components — an outer loop and an innter loop. The outer loop (executed in `proportionality_prior_gradient`) corresponds to the expectation portion of the function, where we want to find time steps where the agent took the same action. 

We then use a helper function, `proportional_loss_gradient`, to compute the gradient of $||\triangle s_{t_2}|| - ||\triangle s_{t_1}||)^2$. Now remember, to compute this gradient, it is useful to think of the function less in terms of the state representation themselves $s$, but rather in terms of the image vectors ($v_{t_1}, v_{t_1+1}, v_{t_2}, v_{t_2+1}$) and the mapping matrix ($m$). Which would give you:

$\frac{d}{dm} (||m v_{t_1 +1} - m v_{t_1}|| - ||m v_{t_2 +1} - m v_{t_2}||)^2$

Which is important to think about when computing gradients because your gradients are always with respect to $m$, which is concealed inside the state representations. This derivative is computer below by iterating over each element of $m$ and computing the gradient of it with respect to the rest of the lost function, and then we return the entire gradient of $m$. 

You can see the gradient computation is grounded, not in the individual value of the state representations, but rather in the delta between state representations, and is focused on measuring the difference between those deltas. In implementing the gradients for the remaining functions, we hope you can refine your mathematical intuition behind the formulation of these loss function components 

In [ ]:
def proportionality_prior_gradient(batch, mapping):
  """
  :param batch an array of Data_Frame objects
  :param mapping the current matrix that maps flattened image vectors to coordinates
  :return the gradient of the proportionality prior loss (should be the same dimensions as the mapping matrix)
  """

  time_steps = len(batch)
  total_loss_grad = np.zeros(mapping.shape)
  for i in range(0, time_steps - 1):
      for j in range(i+1,time_steps - 1):
          if batch[i].action == batch[j].action:
              loss_grad = proportional_loss_gradient(batch[i].image, batch[i + 1].image, batch[j].image, batch[j+1].image, mapping)
              total_loss_grad += loss_grad.reshape(mapping.shape)

  return total_loss_grad / (time_steps - 1)


def proportional_loss_gradient(s1, s2, s3, s4, mapping):
    dims = mapping.shape
    output = np.zeros(dims)
    delta1 = mapping@s2 - mapping@s1
    delta2 = mapping@s4 - mapping@s3
    outest = (np.linalg.norm(delta2) - np.linalg.norm(delta1))*2

    for i in range(0, dims[0]):

        outer_denom_1 = delta1[i]*np.linalg.norm(delta1)
        outer_denom_2 = delta2[i]*np.linalg.norm(delta2)
        for j in range(0, dims[1]):
            frac_1 = (delta1[i]**2)*(s1[j]- s2[j])/outer_denom_1
            frac_2 = (delta2[i]**2)*(s3[j] - s4[j])/outer_denom_2
            output[i,j] = -(frac_2 - frac_1)*outest

    return output


## Part A. Temporal Cohesion

The temporal cohesion loss represents Newton's Law of Inertia by penalizing temporally adjacent timeframes for being too different from each other.

In lecture, we defined the temporal cohesion loss as $E[||\triangle s_t||^2]$, where $s_t$ is the state representation at time $t$.
Use the function skeleton below to implement the gradient of the temporal cohesion loss.

Hint: You will want to take the derivative of $E[||\triangle s_t||^2]$ with respect to the `mapping` matrix, and implement that as part of the code.

In [ ]:
def temporal_cohesion_gradient(batch, mapping):
  """
  :param batch an array of Data_Frame objects
  :param mapping the current matrix that maps flattened image vectors to coordinates
  :return the gradient of the temporal cohesion loss (should be the same dimensions as the mapping matrix)
  """

  raise NotImplementedError("finish this!")

In [ ]:
mapping = np.arange(200).reshape((2, 100))
print(check_solution(temporal_cohesion_gradient, solutions.temporal_cohesion_sol, mapping))

False


## Part B. Causality Prior

The causality prior loss represents Newton's Third Law of equal and opposite reactions by penalizing same actions with different rewards having similar state representations.

In lecture, we defined the proportionality prior loss between two timesteps $t_1$ and $t_2$ as $E[e^{-|| s_{t_2} -  s_{t_1}||} | a_{t_1} = a_{t_2} \wedge r_{t_1} \neq r_{t_2}]$.
Use the function skeleton below to implement the gradient of the causality prior loss.

In [ ]:
def causality_prior_gradient(batch, mapping):
  """
  :param batch an array of Data_Frame objects
  :param mapping the current matrix that maps flattened image vectors to coordinates
  :return the gradient of the causality prior loss (should be the same dimensions as the mapping matrix)
  """

  #TODO: Your code here


  raise NotImplementedError("finish this!")

## Part B. Repeatability Prior

The repeatability prior loss represents Newton's Third Law of equal and opposite reactions by penalizing same actions taken from similar positions for transitioning to different positions.

In lecture, we defined the proportionality prior loss between two timesteps $t_1$ and $t_2$ as $E[e^{-|| s_{t_2} -  s_{t_1}||}||\triangle s_{t_2} - \triangle s_{t_1}||^2  | a_{t_1} = a_{t_2}]$.
Use the function skeleton below to implement the gradient of the repeatability prior loss.

In [ ]:
def repeatability_prior_gradient(batch, mapping):
  """
  :param batch an array of Data_Frame objects
  :param mapping the current matrix that maps flattened image vectors to coordinates
  :return the gradient of the repeatability prior loss (should be the same dimensions as the mapping matrix)
  """

  #TODO: Your code here


  raise NotImplementedError("finish this!")